In [6]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain,SequentialChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [7]:
import openai
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

# Manually set the environment variable from .env, prioritizing it over system-wide variables
openai.api_key = os.getenv('OPENAI_API_KEY')
huggingface_api_key = os.getenv('HUGGINGFACEHUB_API_TOKEN')

# Check to ensure the correct key is loaded
# print(os.getenv('OPENAI_API_KEY'))
# print(os.getenv('AVIATIONSTACK_API_KEY'))


In [9]:
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI  # Importing the updated OpenAI class
from langchain.chains import LLMChain, SequentialChain

# Initialize your LLM client
llm = OpenAI()  # Replace with your actual API key
prompt = PromptTemplate.from_template("What is a good name for a company that manufactures {product}")



# Chain to generate a restaurant name
name_chain = LLMChain(llm=llm, prompt=prompt)
print(name_chain.run("advanced military missiles").strip())


"Precision Defense Technologies"


In [10]:
llm

OpenAI(client=<openai.resources.completions.Completions object at 0x00000127A4397C50>, async_client=<openai.resources.completions.AsyncCompletions object at 0x00000127A476BB50>, model_kwargs={}, openai_api_key=SecretStr('**********'), logit_bias={})

In [11]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    }
}


In [12]:
TEMPLATE = """
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming to the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide.
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""


In [13]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [14]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [15]:
TEMPLATE2 = """
You are an expert English grammarian and writer. Given a Multiple Choice Quiz for {subject} students. 
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis.
If the quiz is not at par with the cognitive and analytical abilities of the students,
update the quiz questions which need to be changed and change the tone such that it perfectly fits the students.
Quiz_MCQS:
{quiz}

Check from an expert English Writer of the above quiz:
"""


In [16]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"], 
    template=TEMPLATE
)


In [17]:
review_chain = LLMChain(
    llm=llm, 
    prompt=quiz_evaluation_prompt, 
    output_key="review", 
    verbose=True
)


In [18]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain], 
    input_variables=["text", "number", "subject", "tone","response_json"], 
    output_variables=["quiz", "review"], 
    verbose=True,
)


In [19]:
file_path=r"C:\Users\Dell\Desktop\Python\Generative AI\MCQs-Generator-Generative-AI-Project\data.txt"

In [20]:
with open(file_path,'r') as file:
    TEXT =file.read()

In [21]:
TEXT

'History\nSee also: Timeline of machine learning\nThe term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers 

In [22]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [35]:
NUMBER=4
SUBJECT= "Machine Learning History"
TONE = "simple"

In [36]:
# https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking
# How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: History
See also: Timeline of machine learning
The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithm

In [37]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")


Total Tokens: 3154
Prompt Tokens: 2662
Completion Tokens: 492
Total Cost: 0.0049770000000000005


In [38]:
response

{'text': 'History\nSee also: Timeline of machine learning\nThe term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other res

In [39]:
quiz = response.get("quiz")

In [40]:
quiz

'\n{"1": {"mcq": "Who coined the term machine learning?", "options": {"a": "John Hopfield", "b": "Donald Hebb", "c": "Arthur Samuel", "d": "Warren McCulloch"}, "correct": "c"}, "2": {"mcq": "In what year did the term machine learning first appear?", "options": {"a": "1949", "b": "1959", "c": "1969", "d": "1979"}, "correct": "b"}, "3": {"mcq": "Which field did machine learning emerge from?", "options": {"a": "Artificial Intelligence", "b": "Computer Science", "c": "Statistics", "d": "All of the above"}, "correct": "d"}, "4": {"mcq": "What was the main focus of machine learning in the 1990s?", "options": {"a": "Achieving artificial intelligence", "b": "Solving practical problems", "c": "Using symbolic approaches", "d": "None of the above"}, "correct": "b"}}'

In [42]:
quiz = json.loads(quiz)

In [43]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})


In [44]:
quiz_table_data

[{'MCQ': 'Who coined the term machine learning?',
  'Choices': 'a: John Hopfield | b: Donald Hebb | c: Arthur Samuel | d: Warren McCulloch',
  'Correct': 'c'},
 {'MCQ': 'In what year did the term machine learning first appear?',
  'Choices': 'a: 1949 | b: 1959 | c: 1969 | d: 1979',
  'Correct': 'b'},
 {'MCQ': 'Which field did machine learning emerge from?',
  'Choices': 'a: Artificial Intelligence | b: Computer Science | c: Statistics | d: All of the above',
  'Correct': 'd'},
 {'MCQ': 'What was the main focus of machine learning in the 1990s?',
  'Choices': 'a: Achieving artificial intelligence | b: Solving practical problems | c: Using symbolic approaches | d: None of the above',
  'Correct': 'b'}]

In [51]:
quiz = pd.DataFrame(quiz_table_data)

In [52]:
quiz.to_csv("machine_learning.csv",index=False)